In [1]:
from pyspark.sql import SparkSession # 라이브러리 사용 

spark = SparkSession.builder.appName("Pyspark Example").getOrCreate() 
spark

24/12/03 11:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#RDD 생성 - 분산데이터객체 
rdd= spark.sparkContext.parallelize( [1,2,3,4,5] ) # - 분산객체관리, 드라이버 영역 안에 객체 생성 
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
rdd.take(5)

[1, 2, 3, 4, 5]

In [4]:
squared_rdd = rdd.map( lambda x : x * x ) #제곱해서 가져오기
squared_rdd

PythonRDD[3] at RDD at PythonRDD.scala:53

In [5]:
squared_rdd.take(5)

[1, 4, 9, 16, 25]

In [ ]:
# 데이터프레임 객체 

In [6]:
data= [ ("Alicce",1), ("Bob",2), ("Charile",3)] # 리스트 값만들기 데이터만들기 
df= spark.createDataFrame(data, ["Name", "Value"]) #객체 꼭 담아주기!! 
df 

DataFrame[Name: string, Value: bigint]

In [7]:
df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
| Alicce|    1|
|    Bob|    2|
|Charile|    3|
+-------+-----+



In [21]:
df.filter('Value > 2').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charile|    3|
+-------+-----+



In [9]:
#RDBMS 데이터 - 테이블 

In [10]:
df.createOrReplaceTempView("people") # 가상 테이블 만듦 

In [11]:
select_sql= "SELECT * FROM people WHERE Value >2"

In [12]:
result_sql= spark.sql(select_sql)
result_sql.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charile|    3|
+-------+-----+



In [ ]:
# DataFrame 생성 -> select DF 추출

In [ ]:
#MLlib

In [18]:
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 154.8 MB/s eta 0:00:00


In [19]:
#MLlib
import numpy as np

In [20]:
#DataFrame 생성 - select DF 추출 

In [23]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression  import LinearRegression



In [24]:
# DataFrame 생성
df = spark.createDataFrame([("Alice", 25), ("Bob", 30), ("Charlie", 35)], ["name", "age"])

# DataFrame 연산
df_filtered = df.filter(df.age > 28)
df_filtered.show()

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [25]:
assembler = VectorAssembler(inputCols=['age'], outputCol='features')
vector_df = assembler.transform(df)#머신러닝 모델만드는 코드 
#예측결과를 보여줌 pred => spark이 연결하는 rdp, -- 에 바로 내보낼 수 있음. 

In [26]:
lr = LinearRegression(featuresCol='features', labelCol='age')
model = lr.fit(vector_df)

24/12/03 11:35:14 WARN Instrumentation: [1b83f9b0] regParam is zero, which might cause numerical instability and overfitting.
24/12/03 11:35:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/03 11:35:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/03 11:35:14 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/03 11:35:14 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [27]:
pred = model.transform(vector_df)
pred.show()

+-------+---+--------+------------------+
|   name|age|features|        prediction|
+-------+---+--------+------------------+
|  Alice| 25|  [25.0]|25.000000000000036|
|    Bob| 30|  [30.0]|30.000000000000004|
|Charlie| 35|  [35.0]| 34.99999999999997|
+-------+---+--------+------------------+



In [29]:
spark.stop()